### Extracting npy from kaldi ark files

In [2]:
!head /home/alexandertchk/VSCode/multimodal/how2-dataset/audio_300/fbank_pitch_181506/raw_fbank_pitch_all_181506.1.scp

--7E2sU6zP4_10 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:15
--7E2sU6zP4_11 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:49458
--7E2sU6zP4_12 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:125776
--7E2sU6zP4_13 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:172983
--7E2sU6zP4_5 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:192669
--7E2sU6zP4_6 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:221557
--7E2sU6zP4_7 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:275815
--7E2sU6zP4_8 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:314937
--7E2sU6zP4_9 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:364164
--8pSDeC-fg_0 ARK_PATH/raw_fbank_pitch_all_181506.1.ark:396664


In [3]:
vid_ids = []
fbank_pitch_181506_path = "/home/alexandertchk/VSCode/multimodal/how2-dataset/audio_300/fbank_pitch_181506"
with open(fbank_pitch_181506_path + '/cmvn_all_181506.scp') as f:
    for line in f:
        vid, path = line.split()
        vid_ids.append(vid)

In [4]:
len(vid_ids)

13668

In [5]:
vid_ids = set(vid_ids)

In [6]:
import sys
import os

"""Replaces paths in Kaldi .scp files with a given placeholder.
Reads from STDIN and writes to STDOUT."""

try:
    pattern = fbank_pitch_181506_path
except IndexError as ie:
    print('Usage: {} <placeholder string>'.format(sys.argv[0]))
    sys.exit(1)

for idx in range(1, 11):
    print("Processing file at idx", idx)
    with open(fbank_pitch_181506_path + '/raw_fbank_pitch_all_181506.{}.scp'.format(idx)) as f, open(fbank_pitch_181506_path + '/raw_fbank_pitch_all_181506.{}_processed.scp'.format(idx), 'w') as fw:
        for line in f:
            line = line.strip()
            utt, path = line.split()
            ark_file, offset = path.split(':')
            ark_file = ark_file.split('/')[-1]
            fw.write('{} {}/{}:{}\n'.format(utt, pattern, ark_file, offset))

Processing file at idx 1
Processing file at idx 2
Processing file at idx 3


Processing file at idx 4
Processing file at idx 5
Processing file at idx 6
Processing file at idx 7
Processing file at idx 8
Processing file at idx 9
Processing file at idx 10


In [7]:
audio_feats = {}

from kaldiio import ReadHelper
for fidx in range(1, 2):
    print("Evaluating file", fidx)
    with ReadHelper(f'scp:{fbank_pitch_181506_path}/raw_fbank_pitch_all_181506.{fidx}_processed.scp') as reader:
        for idx, (key, numpy_array) in enumerate(reader):
            audio_feats[key] = numpy_array

Evaluating file 1


In [8]:
audio_feats

{'--7E2sU6zP4_10': array([[ 1.33435392e+01,  1.34493484e+01,  1.15592442e+01, ...,
          6.57519177e-02, -2.48693824e-02,  4.65756506e-02],
        [ 1.43025455e+01,  1.60679817e+01,  1.69987659e+01, ...,
         -1.93527699e-01, -2.48693824e-02,  1.37654662e-01],
        [ 1.52859964e+01,  1.67500744e+01,  1.98471756e+01, ...,
         -2.42214203e-01,  4.43451107e-02,  1.50330648e-01],
        ...,
        [ 1.24373922e+01,  1.46604671e+01,  1.52252274e+01, ...,
          1.04104526e-01,  7.18828619e-01,  3.59221399e-02],
        [ 1.22473936e+01,  1.43004045e+01,  1.46340475e+01, ...,
          1.27116099e-01,  7.18828619e-01,  1.46151185e-02],
        [ 1.34896917e+01,  1.40385408e+01,  1.35591764e+01, ...,
          9.38771591e-02,  7.18828619e-01,  1.35497674e-02]], dtype=float32),
 '--7E2sU6zP4_11': array([[ 1.14040155e+01,  1.27370586e+01,  1.29804373e+01, ...,
         -1.79172605e-02,  3.51846397e-01,  1.79014653e-02],
        [ 1.24357700e+01,  1.15940180e+01,  1.289775

In [9]:
print(len(audio_feats))

19161


In [10]:
from collections import defaultdict

vid_utts = defaultdict(list)

for key in audio_feats.keys():
    try:
        curr_utt_ids = []
        # To manage utts of format vid_1_2_3
        while True:
            split_idx = key.rfind('_')
            vid_id, utt_id = key[:split_idx], key[split_idx+1:]
            curr_utt_ids.append(utt_id)
            if vid_id in vid_ids:
                for utt_id in curr_utt_ids:
                    vid_utts[vid_id].append(int(utt_id))
                break
            key = vid_id
    except:
        print("Error at key:",key)
        break

In [20]:
vid_utts['--8pSDeC-fg']

[0, 1, 10, 11, 12, 13, 14, 15, 16, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
len(vid_utts)

1327

In [22]:
for key in vid_utts:
    vid_utts[key] = sorted(vid_utts[key])

In [55]:
# 00cp1iGiDw_10_11

# for key in vid_utts.keys():
#     if '_' in key:
#         print(key)
#         print(vid_utts[key])
#         break

--dANj_01AU
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [23]:
combined_feats = defaultdict(list)

for vid_id in vid_utts:
    utt_idx = 0
    key = vid_id
    # To manage utts of format vid_1_2_3
    while utt_idx < len(vid_utts[vid_id]):
        utt_id = vid_utts[vid_id][utt_idx]
        key = '{}_{}'.format(key, utt_id)
        if key in audio_feats:
            combined_feats[vid_id].append(audio_feats[key])
            key = vid_id
        utt_idx += 1

In [25]:
combined_feats['--7E2sU6zP4']

[array([[ 1.39740896e+01,  1.55615225e+01,  1.98078861e+01, ...,
         -6.15804255e-01, -3.11720192e-01, -9.99119282e-02],
        [ 1.44322758e+01,  1.63216934e+01,  2.00885639e+01, ...,
         -5.87061763e-01, -3.60555351e-01, -1.12578273e-02],
        [ 1.29361315e+01,  1.71685963e+01,  1.99482250e+01, ...,
         -5.25301397e-01, -4.09152806e-01, -7.54085034e-02],
        ...,
        [ 1.00570621e+01,  1.14150448e+01,  1.26116648e+01, ...,
         -1.07231200e-01, -6.32701099e-01, -1.37735382e-02],
        [ 1.32934618e+01,  1.33140631e+01,  1.25600824e+01, ...,
          6.78795576e-03, -6.32701099e-01,  1.32073462e-03],
        [ 1.22895346e+01,  1.22577152e+01,  1.29727383e+01, ...,
         -4.07200456e-02, -6.32701099e-01,  5.66664189e-02]], dtype=float32),
 array([[ 1.40340281e+01,  1.42861042e+01,  1.42707176e+01, ...,
         -1.65400267e-01, -3.83546501e-02,  5.90007752e-02],
        [ 1.60767155e+01,  1.67161484e+01,  1.95829086e+01, ...,
         -4.62925076e-0

In [26]:
len(combined_feats)

1323

In [27]:
import numpy as np

concat_feats = {}

for key in combined_feats:
    concat = np.vstack(combined_feats[key])
    concat_feats[key] = concat

In [28]:
print(len(concat_feats))

1323


In [29]:
for key, val in concat_feats.items():
    np.save('/home/alexandertchk/VSCode/multimodal/how2-dataset/audio_300/concat-mast/{}.npy'.format(key), val)

In [4]:
import os

test_vid_ids = set()

for fname in os.listdir('/home/aman_khullar/how2/video_features_300/'):
    test_vid_ids.add(fname[:-4])

In [8]:
len(test_vid_ids.intersection(vid_ids))

13445